<a href="https://colab.research.google.com/github/AI4Bharat/indicTrans/blob/main/IndicTrans_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# create a seperate folder to store everything
!mkdir training
%cd training

/content/training


In [ ]:
# clone the repo for running finetuning
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 432, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 432 (delta 122), reused 105 (delta 105), pack-reused 293
Receiving objects: 100% (432/432), 1.43 MiB | 14.11 MiB/s, done.
Resolving deltas: 100% (248/248), done.
/content/training/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1325, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1325 (delta 84), reused 89 (delta 41), pack-reused 1178
Receiving objects: 100% (1325/1325), 9.57 MiB | 10.51 MiB/s, done.
Resolving deltas: 100% (688/688), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 

In [ ]:
! sudo apt install tree

# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install ./
! pip install xformers
%cd ..

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (133 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 160772 files and directories currently install

In [ ]:
# add fairseq folder to python path
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"
# sanity check to see if fairseq is installed
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [ ]:
## for the latest samanantar dataset v0.3 -> please use this link: https://storage.googleapis.com/samanantar-public/V0.3/source_wise_splits.zip
# This v0.3 dataset has source wise splits to indicate where the data has been collected from
# For preprocessing simplicity we will use v0.2( which just uses raw text files without source information) in this tutorial
# 
# 
#  lets now download the indictrans data v0.2 dataset
! wget https://storage.googleapis.com/samanantar-public/V0.2/data/en2indic/samanatar-en-indic-v0.2.zip



# lets also download the benchmarks for dev and test set

! wget https://storage.googleapis.com/samanantar-public/benchmarks.zip

# training data is organized as en-X folders where each folder contains two text files containing parallel data for en-X lang pair.

# final_data
# ├── en-as
# │   ├── train.as
# │   └── train.en
# ├── en-bn
# │   ├── train.bn
# │   └── train.en
# ├── en-gu
# │   ├── train.en
# │   └── train.gu
# ├── en-hi
# │   ├── train.en
# │   └── train.hi
# ├── en-kn
# │   ├── train.en
# │   └── train.kn
# ├── en-ml
# │   ├── train.en
# │   └── train.ml
# ├── en-mr
# │   ├── train.en
# │   └── train.mr
# ├── en-or
# │   ├── train.en
# │   └── train.or
# ├── en-pa
# │   ├── train.en
# │   └── train.pa
# ├── en-ta
# │   ├── train.en
# │   └── train.ta
# └── en-te
#     ├── train.en
#     └── train.te


! unzip samanatar-en-indic-v0.2.zip

# benchmarks folder consists of all the benchmarks we report in the paper - pmi, ufal-ta, wat2020, wat2021, wmt-news

! unzip benchmarks.zip

^C


--2021-12-18 21:31:57--  https://storage.googleapis.com/samanantar-public/benchmarks.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.160.144, 216.58.196.176, 142.250.71.16, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.160.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7301872 (7.0M) [application/zip]
Saving to: 'benchmarks.zip'

     0K .......... .......... .......... .......... ..........  0%  774K 9s
    50K .......... .......... .......... .......... ..........  1% 2.10M 6s
   100K .......... .......... .......... .......... ..........  2% 2.46M 5s
   150K .......... .......... .......... .......... ..........  2% 2.68M 4s
   200K .......... .......... .......... .......... ..........  3% 1.44M 4s
   250K .......... .......... .......... .......... ..........  4% 2.48M 4s
   300K .......... .......... .......... .......... ..........  4% 3.41M 4s
   350K .......... .......... .......... .......... 

Archive:  samanatar-en-indic-v0.2.zip


  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of samanatar-en-indic-v0.2.zip or
        samanatar-en-indic-v0.2.zip.zip, and cannot find samanatar-en-indic-v0.2.zip.ZIP, period.


Archive:  benchmarks.zip
   creating: benchmarks/
   creating: benchmarks/pmi/
   creating: benchmarks/pmi/en-as/
  inflating: benchmarks/pmi/en-as/dev.as  
  inflating: benchmarks/pmi/en-as/dev.en  
  inflating: benchmarks/pmi/en-as/test.as  
  inflating: benchmarks/pmi/en-as/test.en  
   creating: benchmarks/wat2021-devtest/
  inflating: benchmarks/wat2021-devtest/dev.gu  
  inflating: benchmarks/wat2021-devtest/dev.en  
  inflating: benchmarks/wat2021-devtest/test.bn  
  inflating: benchmarks/wat2021-devtest/dev.bn  
  inflating: benchmarks/wat2021-devtest/test.hi  
  inflating: benchmarks/wat2021-devtest/dev.kn  
  inflating: benchmarks/wat2021-devtest/dev.ta  
  inflating: benchmarks/wat2021-devtest/test.pa  
  inflating: benchmarks/wat2021-devtest/test.en  
  inflating: benchmarks/wat2021-devtest/test.mr  
  inflating: benchmarks/wat2021-devtest/test.kn  
  inflating: benchmarks/wat2021-devtest/dev.ml  
  inflating: benchmarks/wat2021-devtest/test.ta  
  inflating: benchmarks/wat

In [ ]:
# create an experiment dir to store train data, devtest data. 
# This folder will also store vocabulary files (created with subword_nmt for bpe), fairseq bin files (for training), model checkpoints.

# for this example we will be training indic to en translation model. We will name our exp_dir as indic-en-exp
! mkdir indic-en-exp
# copying all the train folders to exp_dir
! cp -r final_data/* indic-en-exp

! mkdir -p indic-en-exp/devtest

# copying all benchmarks to devtest folder in exp_dir
! cp -r benchmarks/* indic-en-exp/devtest

# folder to store combined devtest data (based on the domains you want to test, you can combine multiple benchmarks dev datasets, remove duplicates)
! mkdir -p indic-en-exp/devtest/all

# in this tutorial, for simplicity, we will just use wat2020 devtest for dev and test set
! cp -r indic-en-exp/devtest/wat2020-devtest/* indic-en-exp/devtest/all



In [ ]:
% cd indicTrans

In [ ]:
# prepare_data_joint_training.sh takes experiment dir, src_lang, tgt_lang as input 
# This does preprocessing, building vocab, binarization for joint training

# The learning  and applying vocabulary will take a while if the dataset is huge. To make it faster, run it on a multicore system

! bash prepare_data_joint_training.sh '../indic-en-exp' 'indic' 'en'

In [ ]:
# Training the model

# pls refer to fairseq documentaion to know more about each of these options (https://fairseq.readthedocs.io/en/latest/command_line_tools.html)


# some notable args:
# --max-updates         -> maximum update steps the model will be trained for
# --arch=transformer_4x -> we use a custom transformer model and name it transformer_4x (4 times the parameter size of transformer  base)
# --user_dir            -> we define the custom transformer arch in model_configs folder and pass it as an argument to user_dir for fairseq to register this architechture
# --lr                  -> learning rate. From our limited experiments, we find that lower learning rates like 3e-5 works best for finetuning.
# --max_tokens          -> this is max tokens per batch. You should limit to lower values if you get oom errors.
# --update-freq         -> gradient accumulation steps


!( fairseq-train ../indic-en-exp/final_bin \
--max-source-positions=210 \
--max-target-positions=210 \
--max-update=<max_updates> \
--save-interval=1 \
--arch=transformer_4x \
--criterion=label_smoothed_cross_entropy \
--source-lang=SRC \
--lr-scheduler=inverse_sqrt \
--target-lang=TGT \
--label-smoothing=0.1 \
--optimizer adam \
--adam-betas "(0.9, 0.98)" \
--clip-norm 1.0 \
--warmup-init-lr 1e-07 \
--lr 0.0005 \
--warmup-updates 4000 \
--dropout 0.2 \
--save-dir ../indic-en-exp/model \
--keep-last-epochs 5 \
--patience 5 \
--skip-invalid-size-inputs-valid-test \
--fp16 \
--user-dir model_configs \
--wandb-project <wandb_project_name> \
--update-freq=<grad_accumulation_steps> \
--distributed-world-size <num_gpus> \
--max-tokens <max_tokens_in_a_batch> )